In [ ]:
# If needed, uncomment and run this you can run this line once per spark session.
#%pip install azure-cognitiveservices-speech


# Step 1: Import required libraries
import requests
import azure.cognitiveservices.speech as speechsdk
from notebookutils import mssparkutils



In [ ]:
# Step 2: Define the Lakehouse and local paths
lakehouse_file_path = "{provide the file path in the lakehouse preferrably in abfss:// format}"
local_path = "/tmp/Recording.wav"



In [ ]:
# Step 3: Azure Speech credentials (as provided)
speech_key = "{insert your speech key here}"
speech_region = "{insert your speech region}"
token_url = f"https://{speech_region}.api.cognitive.microsoft.com/sts/v1.0/issuetoken"
stt_url = f"https://{speech_region}.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1"




In [ ]:
# Step 4: Copy the audio file from OneLake to local temp
try:
    print("Copying file from OneLake...")
    mssparkutils.fs.cp(lakehouse_file_path, f"file:{local_path}")
    print("✅ File copied successfully to local path.")
except Exception as e:
    print("❌ Failed to copy file:", e)
    raise
#copy back for inspection during debugging only
#mssparkutils.fs.cp("file:/tmp/Recording.m4a", "lakehouse/YourLakehouse/Files/Recording_copy.m4a")

import os
print("✅ File size on local path:", os.path.getsize(local_path), "bytes")

lakehouse_size = mssparkutils.fs.head(lakehouse_file_path, 1)
print("✅ Lakehouse file accessible (head preview shown)")



In [ ]:
# Step 5: Get a Bearer token from Azure Speech
headers_token = {
    "Ocp-Apim-Subscription-Key": speech_key
}

try:
    print("Requesting Azure Speech token...")
    token_response = requests.post(token_url, headers=headers_token)
    token = token_response.text
    print("✅ Token acquired.")
except Exception as e:
    print("❌ Failed to acquire token:", e)
    raise



In [ ]:
# Step 6: Send audio to Speech-to-Text API
headers_stt = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "audio/m4a",
    "Accept": "application/json"
}

params = {
    "language": "en-US",  # Change as needed
    "format": "simple"
}

try:
    print("Sending audio to Azure Speech-to-Text...")
    with open(local_path, "rb") as audio_file:
        response = requests.post(stt_url, headers=headers_stt, params=params, data=audio_file)

    if response.status_code == 200:
        result = response.json()
        print(response.json())
        print("✅ Transcription Result:", result.get("DisplayText", "(No text returned)"))
    else:
        print("❌ API Error:", response.status_code, response.text)
except Exception as e:
    print("❌ Error calling Azure Speech-to-Text:", e)